# CS 105 Mini Project

**Team**: Thomas Henningson, Josh Pennington, Suhail-Anand Dhilon, Cameron Tsai <br><br>
This is our CS 105 Mini Project.

# Introduction

**Question 1**: What data do you have?

1. What was your GPA after your very first quarter of college? <br>
2. ... <br>

**Question 2**: What would you like to know?

Does taking advanced courses in high school have a postive effect on student's grades, study habits, and outlook on education in college?

# Exploring the Data

uhh

# Hypothesis

uhh

# Testing

uhh